## Transfer learning / fine-tuning

This tutorial will guide you through the process of using _transfer learning_ to learn an accurate image classifier from a small number of training samples. Generally speaking, transfer learning refers to the process of leveraging the knowledge learned in one model for the training of another model. 

More specifically, the process involves taking an existing neural network which was previously trained to good performance on a larger dataset, and using it as the basis for a new model which leverages that previous model's accuracy for a new task. Another name for this procedure is called "fine-tuning" because we will take a previous neural network and fine-tune its weights to the new dataset and task. This method has become extremely popular in recent years to improve the performance of a neural net trained on a small dataset. The intuition is that the new dataset may be too small to achieve good performance itself, but we know that most neural nets trained to learn image features often learn similar features anyway, especially at early layers. This has been largely enabled by the open-sourcing of state-of-the-art models; for the top performing models in image classification, it is common practice now to not only publish the architecture, but to release the weights of the model as well. This lets amateurs use top image classifiers to boost the performance of their own task-specific models.

This guide will go through the process of loading a state-of-the-art, 1000-class image classifier, [VGG16](https://arxiv.org/pdf/1409.1556.pdf) which [won the ImageNet challenge in 2014](http://www.robots.ox.ac.uk/~vgg/research/very_deep/), and using it to train a smaller custom classifier on our own images.

How it works: we will first load VGG16 and remove its final layer, the 1000-neuron softmax classification layer specific to ImageNet, and replace it with a new classification layer for the classes we are training over. We will then "freeze" all the weights in the network except the new ones connecting to the new classification layer, and then re-train the model on our new dataset. By keeping the earlier weights fixed, we get to use the features that were discoverd in the previous model.

We will also compare this method to training a small neural network from scratch on the new dataset, and as we shall see, it will dramatically improve our accuracy. We will do that part first.

This guide requires you to install [keras](http://www.keras.io), if you have not done so already. It is highly recommended to make sure you are using the GPU to train these models, as it will otherwise take much longer to train (however it is still possible to use CPU). If you can use GPU and have Theano as your backend, you should run the following command _before_ importing keras, to ensure it uses the GPU.

    os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"
    
If you are using Tensorflow as the backend, this is unnecessary. 

In [1]:
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

import random
import numpy as np
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5103)


The first step is going to be to load our data. As our example, we will be using the dataset [CalTech-101](http://www.vision.caltech.edu/Image_Datasets/Caltech101/), which contains around 9000 labeled images belonging to 101 object categories. However, we will exclude 5 of the categories which have the most images. This is in order to keep the class distribution fairly balanced (around 50-100) and constrained to a smaller number of images, around 6000. 

If you wish to use your own dataset, it should be aranged in the same fashion to `101_ObjectCategories` with all of the images organized into subfolders, one for each class. In this case, the following cell should load your custom dataset correctly by just replacing `root` with your folder. If you have an alternate structure, you just need to make sure that you load the list `data` where every element is a dict where `x` is the data (a 1-d numpy array) and `y` is the label (an integer). Use the helper function `get_image(path)` to load the image correctly into the array, and note also that the images are being resized to 224x224. This is necessary because the input to VGG16 is a 224x224 RGB image. You do not need to resize them on your hard drive, as that is being done in the code below.

If you have `101_ObjectCategories` in your data folder, the following cell should load all the data.

In [2]:
root = '../data/101_ObjectCategories'
exclude = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']
train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories 
              if c not in [os.path.join(root, e) for e in exclude]]

# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

# load all the images from root folder
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames 
              in os.walk(category) for f in filenames 
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# count the number of classes
num_classes = len(categories)

# randomize the data order
random.shuffle(data)

# create training / validation / test split (70%, 15%, 15%)
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

# separate data for labels
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]

# normalize data
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# convert labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# summary
print("finished loading %d images from %d categories"%(len(data), num_classes))
print("train / validation / test split: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("training data shape: ", x_train.shape)
print("training labels shape: ", y_train.shape)


finished loading 6209 images from 97 categories
train / validation / test split: 4346, 931, 932
('training data shape: ', (4346, 224, 224, 3))
('training labels shape: ', (4346, 97))


If everything worked properly, you should have loaded a bunch of images, and split them into three sets: `train`, `val`, and `test`. The shape of the training data should be (`n`, 224, 224, 3) where `n` is the size of your training set, and the labels should be (`n`, `c`) where `c` is the number of classes (97 in the case of `101_ObjectCategories`. 

Notice that we divided all the data into three subsets -- a training set `train`, a validation set `val`, and a test set `test`. The reason for this is to properly evaluate the accuracy of our classifier. During training, the optimizer uses the validation set to evaluate its internal performance, in order to determine the gradient. The `test` set is always held out from the training algorithm, to avoid overfitting, and is only used at the end to evaluate the final accuracy of our model.

Let's quickly look at a few sample images from our dataset.

In [ ]:



555555555



Before doing the transfer learning, let's first build a neural network from scratch for doing classification on our dataset. This will give us a baseline to compare to our fine-tuned network later.

The network we will construct contains 4 alternating convolutional and max-pooling layers, followed by a [dropout](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) after every other conv/pooling pair. After the last pooling layer, we will attach a fully-connected layer with 256 neurons, another dropout layer, then finally a softmax classification layer for our classes.

Our loss function will be, as usual, `categorical_crossentropy` loss, and our learning algorithm will be adadelta. Various things about this network can be changed to get better performance, perhaps using a larger network or a different optimizer will help, but for the purposes of this notebook, the goal is to just get an understanding of an approximate baseline for comparison's sake, and so it isn't neccessary to spend much time trying to optimize this network.

Upon compiling the network, let's run `model.summary()` to get a snapshot of its layers.

In [63]:
# build the network
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

# compile the model to use categorical cross-entropy loss function and adadelta optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
activation_13 (Activation)   (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 109, 109, 32)      9248      
_________________________________________________________________
activation_14 (Activation)   (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 54, 54, 32)        0         
__________

We've created a medium-sized network with ~1.2 million weights and biases (the parameters). Most of them are leading into the one pre-softmax fully-connected layer "dense_5".

We can now go ahead and train our model for 100 epochs with a batch size of 128. We'll also record its history so we can plot the loss over time later. 

In [64]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=100,
                    validation_data=(x_val, y_val))

Train on 4346 samples, validate on 931 samples
Epoch 1/100
4346/4346 [==============================] - 38s - loss: 4.5407 - acc: 0.0308 - val_loss: 4.4815 - val_acc: 0.0602
Epoch 2/100
4346/4346 [==============================] - 38s - loss: 4.4150 - acc: 0.0686 - val_loss: 4.3017 - val_acc: 0.0945
Epoch 3/100
4346/4346 [==============================] - 38s - loss: 4.2222 - acc: 0.1086 - val_loss: 4.0468 - val_acc: 0.1557
Epoch 4/100
4346/4346 [==============================] - 38s - loss: 3.9835 - acc: 0.1484 - val_loss: 3.7900 - val_acc: 0.1976
Epoch 5/100
4346/4346 [==============================] - 38s - loss: 3.7314 - acc: 0.1855 - val_loss: 3.6591 - val_acc: 0.2578
Epoch 6/100
4346/4346 [==============================] - 38s - loss: 3.5076 - acc: 0.2209 - val_loss: 3.4835 - val_acc: 0.2460
Epoch 7/100
4346/4346 [==============================] - 38s - loss: 3.2847 - acc: 0.2630 - val_loss: 3.2689 - val_acc: 0.2997
Epoch 8/100
4346/4346 [==============================] - 38s - l

Let's plot the loss function the loss over time.

In [66]:
0

0

After 50 epochs, we've got a final validation accuracy of around 50%. We've held out a test set for final evaluation which we do in the following cell.

In [5]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

('Test loss:', 3.7721598874857496)
('Test accuracy:', 0.49463519313304721)


Finally, we see that we have achieved a (top-1) accuracy of around 50%. That's not too bad for 6000 images, considering that if we were to use a naive strategy of taking random guesses, we would have only gotten around 1% accuracy. 

Now we can move on to the main strategy for training an image classifier on our small dataset: by starting with a larger and already trained network.

To start, we will load the VGG16 from keras, which was trained on ImageNet and the weights saved online. If this is your first time loading VGG16, you'll need to wait a bit for the weights to download from the web. Once the network is loaded, we can inspect the layers with the `summary()` method again.

In [6]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Notice that VGG16 is much bigger than the network we constructed. It contains 13 convolutional layers and two fully connected layers at the end, and has over 138 million parameters, around 100 times more parameters than the network we made above. Like our first network, a large proportion of the weights are stored in the connections leading into the first fully-connected layer.

VGG16 was made to solve ImageNet, and achieves a [8.8% top-5 error rate](https://github.com/jcjohnson/cnn-benchmarks), which means that 91.2% of test samples were classified correctly within the top 5 predictions for each image. It's top-1 accuracy--equivalent to the accuracy metric we've been using (that the top prediction is correct)--is 73%. This is especially impressive since there are not just 97, but 1000 classes, meaning that random guesses would get us only 0.1% accuracy.

In order to use this network for our task, we remove the final classification layer, the 1000-neuron softmax layer at the end, which corresponds to ImageNet, and instead replace it with a new softmax layer for our dataset, which contains 97 neurons in the case of the 101_ObjectCategories dataset. 

In terms of implementation, it's easier to simply create a copy of VGG from its input layer until the second to last layer, and then work with that, rather than modifying the VGG object directly. So technically we never remove anything, we just circumvent/ignore it. This can be done in the following way, by using the keras `Model` class to initialize a new model whose input layer is the same as VGG but whose output layer is our new softmax layer, called `new_classification_layer`. Note: although it appears we are duplicating this large network, internally Keras is actually just copying all the layers by reference, and thus we don't need to worry about overloading the memory.

In [41]:
# make a reference to VGG's input layer
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)


We are going to retrain this network on the new classifications. But first, we need to freeze the weights and biases in all the layers in the network, except our new one at the end. The point is that the features that were learned in VGG should still be fairly relevant to the new image classification task. Not perfectly optimal, but most likely better than what we can find in our limited dataset. By setting the `trainable` flag in each layer false (except our new classification layer), we ensure all the weights and biases in those layers remain fixed, and we simply "fine-tune" the parameters in the one layer at the end. 

Note that sometimes you don't have to freeze all the pre-classification layers. If you have enough data, it may be desirable to retrain more than just the one layer at the end. In our case, we don't have enough samples for this to work very well, so we simply go ahead with the plan.

After freezing the other layers, we compile the model with exactly the same optimizer and loss function as in our first network, for the sake of a fair comparison. We then run `summary` again to look at the network's architecture.

In [42]:
# make
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True
    
model_new.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model_new.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Looking at the summary, we see the network is identical to the vgg model we instantiated earlier, except the last layer, formerly a 1000-neuron softmax, has been replaced by a new 97-neuron softmax. Additionally, we still have roughly 134 million weights, but now the vast majority of them are "non-trainable params" because we froze the layers they are contained in. We now only have 397,000 trainable parameters, which is actually only a fourth of the number of parameters needed to train the first model.

As before, we go ahead and train the new model, using the same hyperparametrs (batch size and number of epochs) as before, along with the same optimization algorithm. We also keep track of its history as we go.

In [54]:
history = model_new.fit(x_train, y_train, 
                        batch_size=128, 
                        epochs=100, 
                        validation_data=(x_val, y_val))


Train on 4346 samples, validate on 931 samples
Epoch 1/100
4346/4346 [==============================] - 36s - loss: 4.2942 - acc: 0.0900 - val_loss: 3.7752 - val_acc: 0.1472
Epoch 2/100
4346/4346 [==============================] - 36s - loss: 3.4561 - acc: 0.2584 - val_loss: 3.2105 - val_acc: 0.3212
Epoch 3/100
4346/4346 [==============================] - 36s - loss: 2.9204 - acc: 0.3684 - val_loss: 2.7843 - val_acc: 0.3910
Epoch 4/100
4346/4346 [==============================] - 36s - loss: 2.5652 - acc: 0.4473 - val_loss: 2.5515 - val_acc: 0.4823
Epoch 5/100
4346/4346 [==============================] - 36s - loss: 2.2863 - acc: 0.5166 - val_loss: 2.2879 - val_acc: 0.4791
Epoch 6/100
4346/4346 [==============================] - 36s - loss: 2.0762 - acc: 0.5511 - val_loss: 2.1597 - val_acc: 0.5188
Epoch 7/100
4346/4346 [==============================] - 36s - loss: 1.9124 - acc: 0.5874 - val_loss: 1.9776 - val_acc: 0.5585
Epoch 8/100
4346/4346 [==============================] - 36s - l

In [57]:
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)


('Test loss:', 0.80789021463353239)
('Test accuracy:', 0.78218884120171672)


In [58]:
print(history.history['loss'])
print(history.history['acc'])

print(history.history['val_loss'])
print(history.history['val_acc'])




[4.2941599179674723, 3.4560996727958782, 2.9204496808497522, 2.5652155076310867, 2.2863048170768221, 2.0761643421578615, 1.9123944168643803, 1.7626441612915131, 1.6470325468215206, 1.5567548332596177, 1.461559222142816, 1.3878308535719379, 1.3259043960158747, 1.269991418978502, 1.2155848767863844, 1.1613779493152849, 1.1213186751627671, 1.0822984447031596, 1.0497851623843686, 1.0119208326495521, 0.97815125434562133, 0.94910008806925572, 0.93093975625496772, 0.89004653396005184, 0.87818294110739226, 0.85566686563540095, 0.82809179634527841, 0.80655441910124459, 0.79747256020823765, 0.77239716209358078, 0.75532895721601445, 0.73794520611703751, 0.72564979678631047, 0.70633917750710995, 0.69781448009427771, 0.67514473449454238, 0.66999329754793857, 0.65574898209047383, 0.64311635812416201, 0.63145963428649166, 0.62019149704677301, 0.6089866509428784, 0.59779670067882407, 0.58700551171484927, 0.57532888925190739, 0.56747167057229586, 0.55867116395306926, 0.55548610612647187, 0.538323119632

Improving results

Can improve by doing augmentation on data
running longer, trying different optimizers and hyperparams (keep val set)

compare to imagenet results


In [ ]:



import matplotlib.pyplot
from matplotlib.pyplot import imshow

matplotlib.pyplot.switch_backend('agg')

matplotlib.pyplot.figure(figsize=(16,4))
matplotlib.pyplot.plot(np.array([1,2,3,4,5]))
matplotlib.pyplot.show()

history.history['val_acc']